In [ ]:
# Importar librerías necesarias
import pandas as pd
import pickle

# Importar el soporte biblioteca
import sys
sys.path.append("../")
import src.biblioteca as bb

In [ ]:
# Cargar datos de demanda desde un archivo CSV
df = pd.read_csv("../data/scrap/demanda.csv")

# Convertir la columna "datetime" a formato de fecha y hora con zona horaria UTC
df["datetime"] = pd.to_datetime(df["datetime"], utc = True, format = "%Y-%m-%d %H:%M:%S%z")

# Convertir la zona horaria de la columna "datetime" a "Europe/Paris"
df["datetime"] = df["datetime"].dt.tz_convert("Europe/Paris")

# Renombrar las columnas del DataFrame para mayor claridad
df.rename(columns = {"value": "Demanda en MW", "datetime": "Fecha"}, inplace = True)

# Guardar el DataFrame como un archivo pickle para visualización posterior
with open(f"../data/visualizacion/px_demanda.pkl", "wb") as demanda:
    pickle.dump(df, demanda)

In [ ]:
# Cargar datos de demanda en tiempo real desde un archivo CSV
df2 = pd.read_csv("../data/scrap/demanda_tiempo_real.csv")

# Eliminar columnas innecesarias del DataFrame
df2.drop(["percentage_Demanda real", "percentage_Demanda programada", "percentage_Demanda prevista"], axis = 1, inplace = True)

# Convertir la columna "datetime" a formato de fecha y hora con zona horaria UTC
df2["datetime"] = pd.to_datetime(df2["datetime"], utc = True, format = "%Y-%m-%d %H:%M:%S%z")

# Convertir la zona horaria de la columna "datetime" a "Europe/Paris"
df2["datetime"] = df2["datetime"].dt.tz_convert("Europe/Paris")

# Renombrar las columnas del DataFrame para mayor claridad
df2.rename(columns = {"value_Demanda real": "Demanda real en MW", "value_Demanda programada": "Demanda programada en MW",
                    "value_Demanda prevista": "Demanda prevista en MW", "datetime": "Fecha"}, inplace = True)

# Guardar el DataFrame como un archivo pickle para visualización posterior  
with open(f"../data/visualizacion/px_demanda_real.pkl", "wb") as demanda_real:
    pickle.dump(df2, demanda_real)

In [ ]:
# Cargar datos de demanda en tiempo real desde un archivo CSV
df3 = pd.read_csv("../data/scrap/demanda_tiempo_real.csv")

# Eliminar columnas innecesarias del DataFrame
df3.drop(["percentage_Demanda real", "percentage_Demanda programada", "percentage_Demanda prevista"], axis = 1, inplace = True)

# Convertir la columna "datetime" a formato de fecha y hora con zona horaria UTC
df3["datetime"] = pd.to_datetime(df3["datetime"], utc = True, format = "%Y-%m-%d %H:%M:%S%z")

# Crear una nueva columna "drop" que contiene "drop" para filas cuya columna "datetime" no cumple la condición de que su minuto sea divisible por 10, de lo contrario, contiene "datetime"
df3["drop"] = df3["datetime"].where(df3["datetime"].dt.minute % 10 == 0, other = "drop")

# Eliminar las filas cuya columna "drop" tiene el valor "drop"
df3 = df3[df3["drop"] != "drop"]

# Eliminar la columna "drop" del DataFrame
df3.drop(["drop"], inplace = True, axis = 1)

# Crear un nuevo DataFrame vacío para almacenar los datos de demanda por estaciones
df_estaciones = pd.DataFrame()

# Iterar sobre las estaciones y sus valores del diccionario "bb.estaciones"
for k, v in bb.estaciones.items():

    # Crear un nuevo DataFrame para almacenar los datos de la estación en un año determinado
    estacion_todo = pd.DataFrame()

    # Iterar sobre los años desde 2013 hasta 2023
    for year in range(2013, 2024):

        # Crear las fechas de inicio y fin basadas en los valores de la estación y el año actual
        inicio = pd.to_datetime(f"{year}-{v[0]} 00:00:00+01:00")

        if k == "Invierno":
            fin = pd.to_datetime(f"{year + 1}-{v[1]} 23:59:59+02:00")

        else:
            fin = pd.to_datetime(f"{year}-{v[1]} 23:59:59+02:00")

        # Filtrar los datos de demanda en tiempo real para el rango de fechas actual
        df_estacion_año = df3[(df3["datetime"] >= inicio) & (df3["datetime"] <= fin)]

        # Concatenar los datos filtrados al DataFrame de la estación para el año actual
        estacion_todo = pd.concat([estacion_todo, df_estacion_año], axis = 0)

    # Crear columnas de día de la semana, hora y minuto en el DataFrame de la estación
    estacion_todo["dia"] = estacion_todo["datetime"].dt.day_of_week
    estacion_todo["hour"] = estacion_todo["datetime"].dt.hour
    estacion_todo["minute"] = estacion_todo["datetime"].dt.minute

    # Calcular la demanda promedio por día de la semana, hora y minuto para la estación actual
    estacion_semana = estacion_todo.groupby(by = ["dia", "hour", "minute"])["value_Demanda real"].mean().reset_index()

    # Crear un nuevo DataFrame con los valores de demanda promedio para la estación actual
    estacion = pd.DataFrame(estacion_semana["value_Demanda real"])

    # Renombrar la columna de demanda con el nombre de la estación
    estacion.columns = [f"Demanda {k}"]

    # Concatenar los datos de demanda de la estación actual al DataFrame de todas las estaciones
    df_estaciones = pd.concat([df_estaciones, estacion], axis = 1, ignore_index = True)

# Renombrar las columnas del DataFrame para mayor claridad
df_estaciones.columns = ["Primavera", "Verano", "Otoño", "Invierno"]

# Se crea una nueva columna en el DataFrame de demanda por estación llamada "Día" que contiene el número de día en orden ascendente
df_estaciones["Día"] = [n+1 for n in range(df_estaciones.shape[0])]

# Guardar el DataFrame como un archivo pickle para visualización posterior  
with open(f"../data/visualizacion/px_demanda_estacion.pkl", "wb") as demanda_estacion:
    pickle.dump(df_estaciones, demanda_estacion)

In [ ]:
# Lee el archivo CSV de precios de mercado
df4 = pd.read_csv("../data/scrap/precios_mercados.csv")

# Convierte la columna "datetime" a formato de fecha y hora con zona horaria UTC
df4["datetime"] = pd.to_datetime(df4["datetime"], utc = True, format = "%Y-%m-%d %H:%M:%S%z")

# Convierte la zona horaria de la columna "datetime" a "Europe/Paris"
df4["datetime"] = df4["datetime"].dt.tz_convert("Europe/Paris")

# Crea una copia del DataFrame original
df_diario = df4.copy()

# Renombra las columnas del DataFrame copiado
df_diario.rename(columns = {"value": "Precio mayorista en €/MWh", "datetime": "Fecha"}, inplace = True)

# Guarda el DataFrame copiado en un archivo pickle
with open(f"../data/visualizacion/px_precio_diario.pkl", "wb") as precios_diario:
    pickle.dump(df_diario, precios_diario)

# Crea una nueva columna con el mes y año en formato "mm-YYYY"
df4["mes_y_año"] = df4["datetime"].dt.strftime("%m-%Y")

# Agrega por la columna "mes_y_año" y suma la columna "value"
df_mes = (df4.groupby(by = "mes_y_año")["value"].sum()).reset_index()

# Convierte la columna "mes_y_año" a formato de fecha
df_mes["mes_y_año"] = pd.to_datetime(df_mes["mes_y_año"], format = "%m-%Y")

# Ordena el DataFrame por la columna "mes_y_año" en orden ascendente
df_mes.sort_values(by = "mes_y_año", ascending = True, inplace = True)

# Renombra las columnas del DataFrame de agregación
df_mes.rename(columns = {"value": "Precio mayorista en €/MWh", "mes_y_año": "Fecha"}, inplace = True)

# Guarda el DataFrame de agregación en un archivo pickle
with open(f"../data/visualizacion/px_precio_historico.pkl", "wb") as precios_historico:
    pickle.dump(df_mes, precios_historico)

In [ ]:
# Leer archivo CSV
df5 = pd.read_csv("../data/scrap/balance.csv")

# Convertir columna "datetime" a tipo de dato datetime con zona horaria en UTC
df5["datetime"] = pd.to_datetime(df5["datetime"], utc = True, format = "%Y-%m-%d %H:%M:%S%z")

# Convertir zona horaria a Europa/Paris
df5["datetime"] = df5["datetime"].dt.tz_convert("Europe/Paris")

# Extraer el año de la columna "datetime"
df5["año"] = df5["datetime"].dt.year

# Crear nuevo DataFrame con las columnas "datetime", "value_Generación no renovable" y "value_Generación renovable"
df_generado = df5[["datetime", "value_Generación no renovable", "value_Generación renovable"]]

# Calcular la generación total en MW sumando la generación no renovable y renovable
df_generado["Generación en MW"] = df_generado["value_Generación no renovable"] + df_generado["value_Generación renovable"]

# Crear una columna "mes_y_año" en formato MM-YYYY
df_generado["mes_y_año"] = df_generado["datetime"].dt.strftime("%m-%Y")

# Calcular el promedio mensual de la generación total en MW
df_generado_mes = (df_generado.groupby(by = "mes_y_año")["Generación en MW"].mean()).reset_index()

# Convertir la columna "mes_y_año" a tipo de dato datetime en formato MM-YYYY
df_generado_mes["mes_y_año"] = pd.to_datetime(df_generado_mes["mes_y_año"], format="%m-%Y")

# Ordenar por fecha en orden ascendente
df_generado_mes.sort_values(by = "mes_y_año", ascending = True, inplace = True)

# Renombrar columna "mes_y_año" a "Fecha"
df_generado_mes.rename(columns = {"mes_y_año": "Fecha"}, inplace = True)

# Guardar DataFrame en un archivo pickle
with open(f"../data/visualizacion/px_generado.pkl", "wb") as generado:
    pickle.dump(df_generado_mes, generado)

# Calcula el balance de generación de energía no renovable
balance_no_reno = df5[["percentage_Nuclear", "percentage_Ciclo combinado", "percentage_Carbón", "percentage_Turbina de gas", "percentage_Motores diésel", "percentage_Turbina de vapor", "percentage_Fuel + Gas", "percentage_Cogeneración","percentage_Residuos no renovables"]] * 60.25
balance_no_reno.columns = [energia.replace("percentage_", "") for energia in balance_no_reno.columns]

# Calcula el promedio de generación de energía no renovable por tipo de tecnología
no_renovables = balance_no_reno.fillna(0).mean().reset_index(name = "value").rename(columns = {"index": "Energia"})
no_renovables["Tipo"] = "No Renovable"

# Calcula el balance de generación de energía renovable
balance_reno = df5[["percentage_Turbinación bombeo", "percentage_Hidráulica", "percentage_Eólica", "percentage_Solar fotovoltaica", "percentage_Solar térmica", "percentage_Otras renovables", "percentage_Residuos renovables", "percentage_Hidroeólica"]] * 39.75
balance_reno.columns = [energia.replace("percentage_", "") for energia in balance_reno.columns]

# Calcula el promedio de generación de energía renovable por tipo de tecnología
renovables = balance_reno.fillna(0).mean().reset_index(name = "value").rename(columns = {"index": "Energia"})
renovables["Tipo"] = "Renovable"

# Concatena los DataFrames de renovables y no renovables
energias = pd.concat([renovables, no_renovables], axis = 0)
energias["value"] = round(energias["value"], 2)
energias.sort_values(by = "value", ascending=False, inplace = True)
energias.rename(columns = {"value": "Porcentaje"}, inplace = True)

# Guarda el DataFrame en un archivo pickle
with open(f"../data/visualizacion/px_porcentage_renovables.pkl", "wb") as porcentage_renovables:
    pickle.dump(energias, porcentage_renovables)

# Seleccionar las columnas de interés del DataFrame df5
todas_energias = df5[["datetime","value_Turbinación bombeo","value_Nuclear", "value_Ciclo combinado", "value_Carbón", "value_Turbina de gas", "value_Motores diésel", "value_Turbina de vapor", "value_Fuel + Gas", "value_Cogeneración","value_Residuos no renovables", "value_Hidráulica","value_Eólica", "value_Solar fotovoltaica", "value_Solar térmica", "value_Otras renovables", "value_Residuos renovables", "value_Hidroeólica"]]

# Renombrar las columnas para quitar el prefijo "value_"
todas_energias.columns = ["datetime"] + [energia.replace("value_", "") for energia in todas_energias.drop("datetime", axis = 1).columns]

# Rellenar los valores nulos con cero
todas_energias.fillna(0, inplace = True)

# Crear una nueva columna con el formato de mes y año
todas_energias["mes_y_año"] = todas_energias["datetime"].dt.strftime("%m-%Y")

# Agrupar los datos por mes y año, y calcular el promedio de las columnas de energía
todas_energias_mes = todas_energias.groupby(by = "mes_y_año")[bb.tipos_energia].mean().reset_index()

# Convertir la columna de mes y año a formato datetime
todas_energias_mes["mes_y_año"] = pd.to_datetime(todas_energias_mes["mes_y_año"], format = "%m-%Y")

# Ordenar los datos por fecha en orden ascendente
todas_energias_mes.sort_values(by = "mes_y_año", ascending = True, inplace = True)

# Renombrar la columna de mes y año a "Fecha"
todas_energias_mes.rename(columns = {"mes_y_año": "Fecha"}, inplace = True)

# Guardar los resultados en un archivo pickle
with open(f"../data/visualizacion/px_balance_renovables.pkl", "wb") as balance_renovables:
    pickle.dump(todas_energias_mes, balance_renovables)

In [ ]:
# Leer el archivo CSV en un DataFrame df6
df6 = pd.read_csv("../data/scrap/evolucion_renovable_no_renovable.csv")

# Convertir la columna "datetime" a formato datetime con información de zona horaria en UTC
df6["datetime"] = pd.to_datetime(df6["datetime"], utc = True, format = "%Y-%m-%d %H:%M:%S%z")

# Convertir la zona horaria de la columna "datetime" a "Europe/Paris"
df6["datetime"] = df6["datetime"].dt.tz_convert("Europe/Paris")

# Calcular el promedio del porcentaje de energía renovable y no renovable
porcentaje_reno = round(df6["percentage_Renovable"].mean() * 100, 2)
porcentaje_no_reno = round(df6["percentage_No renovable"].mean() * 100, 2)

# Crear una nueva columna con el formato de mes y año
df6["mes_y_año"] = df6["datetime"].dt.strftime("%m-%Y")

# Agrupar los datos por mes y año, y calcular el promedio de los porcentajes de energía renovable y no renovable
evolucion_reno_mes = df6.groupby(by="mes_y_año")["percentage_Renovable", "percentage_No renovable"].mean().reset_index()

# Convertir la columna de mes y año a formato datetime
evolucion_reno_mes["mes_y_año"] = pd.to_datetime(evolucion_reno_mes["mes_y_año"], format = "%m-%Y")

# Ordenar los datos por fecha en orden ascendente
evolucion_reno_mes.sort_values(by = "mes_y_año", ascending = True, inplace = True)

# Guardar los resultados en un archivo pickle
with open(f"../data/visualizacion/px_evo_reno.pkl", "wb") as evo_reno:
    pickle.dump(evolucion_reno_mes, evo_reno)

In [ ]:
# Leer el archivo CSV en un DataFrame df7
df7 = pd.read_csv("../data/scrap/emisiones_CO2.csv")

# Convertir la columna "datetime" a formato datetime con información de zona horaria en UTC
df7["datetime"] = pd.to_datetime(df7["datetime"], utc = True, format = "%Y-%m-%d %H:%M:%S%z")

# Convertir la zona horaria de la columna "datetime" a "Europe/Paris"
df7["datetime"] = df7["datetime"].dt.tz_convert("Europe/Paris")

# Crear una nueva columna con el formato de mes y año
df7["mes_y_año"] = df7["datetime"].dt.strftime("%m-%Y")

# Agrupar los datos por mes y año, y sumar los valores de las columnas de emisiones de CO2
df_mes_emisiones = df7.groupby(by="mes_y_año")["value_Carbón", "value_Motores diésel", "value_Turbina de gas", "value_Turbina de vapor", "value_Ciclo combinado",
                                                "value_Cogeneración", "value_Residuos no renovables"].sum().reset_index()

# Convertir la columna de mes y año a formato datetime
df_mes_emisiones["mes_y_año"] = pd.to_datetime(df_mes_emisiones["mes_y_año"], format = "%m-%Y")

# Ordenar los datos por fecha en orden ascendente
df_mes_emisiones.sort_values(by = "mes_y_año", ascending = True, inplace = True)

# Renombrar las columnas para hacerlas más descriptivas
df_mes_emisiones.rename(columns = {"mes_y_año": "Fecha", "value_Carbón": "Carbón", "value_Motores diésel": "Motores diésel","value_Turbina de gas": "Turbina de gas",
                                    "value_Turbina de vapor": "Turbina de vapor", "value_Ciclo combinado": "Ciclo combinado", "value_Cogeneración": "Cogeneración",
                                    "value_Residuos no renovables": "Residuos no renovables"}, inplace = True)

# Guardar los resultados en un archivo pickle
with open(f"../data/visualizacion/px_emisiones.pkl", "wb") as emisiones:
    pickle.dump(df_mes_emisiones, emisiones)


In [ ]:
# Leer archivo CSV y cargar datos en un DataFrame (df8)
df8 = pd.read_csv("../data/scrap/perdidas_transporte.csv")

# Convertir la columna "datetime" en tipo datetime y ajustar a la zona horaria de Europa/París
df8["datetime"] = pd.to_datetime(df8["datetime"], utc = True, format = "%Y-%m-%d %H:%M:%S%z")
df8["datetime"] = df8["datetime"].dt.tz_convert("Europe/Paris")

# Multiplicar la columna "percentage" por 100 para convertir a porcentaje
df8["percentage"] = (df8["percentage"] * 100)

# Crear una nueva columna "mes_y_año" con el mes y año correspondiente en formato de texto
df8["mes_y_año"] = df8["datetime"].dt.strftime("%m-%Y")

# Agrupar por mes y año, y calcular el promedio de la columna "percentage"
perdidas2_mes = (df8.groupby(by = "mes_y_año")["percentage"].mean()).reset_index()

# Convertir la columna "mes_y_año" a tipo datetime
perdidas2_mes["mes_y_año"] =  pd.to_datetime(perdidas2_mes["mes_y_año"], format = "%m-%Y")

# Ordenar por fecha en orden ascendente
perdidas2_mes.sort_values(by = "mes_y_año", ascending = True, inplace = True)

# Renombrar columnas
perdidas2_mes.rename(columns = {"mes_y_año" : "Fecha","percentage": "Porcentaje"}, inplace = True)

# Guardar el DataFrame en un archivo pickle
with open(f"../data/visualizacion/px_perdidas.pkl", "wb") as perdidas:
    pickle.dump(perdidas2_mes, perdidas)

In [ ]:
# Leer archivo CSV y cargar datos en un DataFrame (df_reno_ccaa)
df_reno_ccaa = pd.read_csv("../data/scrap/evolucion_renovable_no_renovable_ccaa.csv")

# Mapear los valores de la columna "ccaa" utilizando un diccionario del soporte biblioteca
df_reno_ccaa["ccaa"] = df_reno_ccaa["ccaa"].map(bb.ccaa_real)

# Guardar el DataFrame en un nuevo archivo CSV en la carpeta "../data/visualizacion/" 
df_reno_ccaa.to_csv("../data/visualizacion/reno_ccaa.csv")